## ARC 380 / CEE 380 / ROB 380 – Introduction to Robotics for Digital Fabrication  
### Session 8 Workshop: Robot Description, Visualization, and Simulation in ROS  
Princeton University, Spring 2026  
Professor: Arash Adel | TA: Daniel Ruan

---

# Overview

By the end of this workshop you should be able to:

1. Explain what a **ROS 2 package** is and why we use **colcon** workspaces.
2. Read a robot description package (URDF + meshes) and launch a basic visualization.
3. Use **RViz2** to visualize a robot and **joint_state_publisher_gui** to jog joints.
4. Implement a small **rclpy** node that publishes `sensor_msgs/JointState` using **runtime parameters**.


Clone the course repository from GitHub:

```bash
git clone https://github.com/AdelResearchGroup/arc380_s26.git
```

# 1. ROS 2 packages and colcon

## 1.1 What is a ROS 2 package?

A **package** is the smallest unit of build + distribution in ROS. It typically contains:
- source code (`.py` or `.cpp`)
- metadata (`package.xml`)
- build / install rules (`setup.py` for Python, `CMakeLists.txt` for C++)

Other contents that we will typical encounter:
- launch files (`launch/`)
- source code (`src/`)
- configuration files (`config/`)
- robot description files (`urdf/`, `meshes/`)

Why packages matter:
- consistent dependency tracking
- reproducible builds
- discoverable executables (`ros2 run <pkg> <exe>`, `ros2 launch <pkg> <launch_file>`)
- sharing + reusing code and robot models


## 1.2 Colcon workspace anatomy

A **colcon workspace** is usually:

```text
~/arc380_s26/          # The workspace root
  src/                 # Packages live here
    some_pkg/
    another_pkg/
  build/               # Generated (intermediate build files, do not edit)
  install/             # Generated (installed packages, what you "source")
  log/                 # Generated (build logs)
```

### 1.2.1 Building + sourcing:

After creating a package (and adding/editing any files), you need to **build** and **source** the package.

Make sure that you are in the workspace root when building. You can use `cd <path>/<to>/arc380_s26` to change your directory to the workspace root, or use `cd ..` to move back up one directory if you are in a subdirectory (e.g., `src` after creating a package).

**Build:**
```bash
colcon build
```
What this command does is:
- Scans the `src/` directory for packages
- Reads each `package.xml` and `setup.py`
- Installs Python modules into the `install/` directory
- Register the package in the **ament index**
- Creates executable entry points defined in `setup.py`

**Source:**
After building, you must "source" the workspace. Only run **one** of the following commands for sourcing, depending on your system + terminal:

```bash
call install/setup.bat      # cmd (Windows command prompt)
source install/setup.bash   # bash (MacOS/Linux bash terminal)
source install/setup.zsh    # zsh (MacOS zsh terminal)
```

What sourcing does is:
- Adds your workspace's `install/` directory to the environment
- Updates environment variables such as `PYTHONPATH`, `AMENT_PREFIX_PATH`, `PATH`
- Makes ROS 2 aware of your built packages

If you do not source the workspace:
- ROS 2 will not be able to find your built packages
- `ros2 run` will fail with "package not found"

You must re-source your workspace:
- In every new terminal
- After rebuilding


## 1.3 Creating a minimal Python package

From your workspace root:

```bash
cd src      # Change Directory (cd) to the src folder

# Create a Python package with ament_python build type
ros2 pkg create --build-type ament_python arc380_session08 --dependencies rclpy
```

What this command generates (typical):
- `package.xml` (dependencies)
- `setup.py`, `setup.cfg`
- `arc380_session08/` Python module directory
- `resource/arc380_session08` (ament index marker)

### 1.3.1 Adding a Hello World script
Custom Python scripts typically go in the `arc380_session08/arc380_session08` directory.

Create a new Python file named `my_script.py` under `arc380_session08/arc380_session08` with a simple `main` function:
```python
# arc380_session08/arc380_session08/my_script.py
def main():
    print("Hello ROS 2!")

```


## 1.4 Running executables

For Python packages, `ros2 run` launches Python entry points defined in `setup.py`. Below, we create an entry point that automatically runs the `main()` function in `my_script.py`.

Define an entry point named `hello_ros` in `setup.py`:

```python
# arc380_session08/setup.py
...
entry_points={
    "console_scripts": [
        "hello_ros = arc380_session08.my_script:main",
    ],
}
```

Since we made changes to the package, we need to build and source the workspace again (see Section 1.2.1).

After building and sourcing, you can now run:

```bash
ros2 run arc380_session08 hello_ros
```

which should print out `Hello ROS 2!` into the terminal.

### 1.4.1 Why use entry points?

Instead of directly calling the script using Python (e.g., `python src/arc380_session08/arc380_session08/my_script.py`), using ROS 2 entry points with `ros2 run` is preferred for several reasons:
- **Package integration**: `ros2 run` ensures your node is executed within the context of the installed ROS 2 package, using the correct environment and dependencies.
- **Consistent workflow**: All ROS nodes (Python and C++) are launched the same way, which keeps development consistent across projects.
- **Dependency resolution**: Entry points rely on the installed package metadata, so imports and package paths are resolved correctly.
- **Scalability**: Larger projects often contain multiple executables. Entry points provide a clean way to expose multiple nodes from a single package.
- **Professional practice**: In real ROS development, nodes are run via `ros2 run` or launch files, not by manually calling Python scripts.

For quick testing during development, running a script directly can be more convenient. However, your final package should always build successfully and run via `ros2 run` to ensure that the node behaves correctly as part of the ROS 2 ecosystem.


---

# 2. Robot description packages

A **robot description package** typically contains:
- `urdf/` (URDF or Xacro)
- `meshes/visual/` (what you see)
- `meshes/collision/` (simplified geometry for collision checking)
- `launch/` (RViz + robot_state_publisher launches)
- sometimes `config/` (RViz configs)

## 2.1 URDF
**URDF** (Unified Robot Description Format) is an XML-based format used in ROS to describe a robot’s kinematic structure and physical properties. It defines:
- The robot’s rigid bodies (links)
- The connections between them (joints)
- The geometric representation (meshes or primitives)
- Optional inertial properties (for physics simulation)

URDF **does not**:
- Perform forward kinematics
- Simulate physics
- Plan motion

Instead, it defines the static model that other ROS nodes use.

In ROS visualization, the URDF is usually published on the `robot_description` parameter by `robot_state_publisher`.


## 2.2 Xacro

In practice, many robot description files are not written as raw `.urdf` files. Instead, they are written in Xacro.

Xacro (XML Macro Language) is a preprocessor for URDF. It allows you to:
- Define reusable macros
- Use parameters
- Define variables
- Avoid repetitive XML
- Create configurable robot variants

A `.xacro` file is converted into a standard `.urdf` file before being used.

For example:
```bash
xacro irb120_3_60_macro.xacro > irb120.urdf
```

The resulting expanded URDF is what `robot_state_publisher` actually consumes.

## 2.3 The `<robot>` root element

Every URDF/Xacro starts with:

```xml
<robot name="irb120">
  ...
</robot>
```

Everything (links, joints, materials, macros) lives inside this root element.

## 2.4 Links

A `<link>` represents a rigid body in the robot.

**Basic Structure**

```xml
<link name="link_1">
  <visual>...</visual>
  <collision>...</collision>
  <inertial>...</inertial>
</link>
```

Each link can define three different “aspects”:
- Visual element
- Collision element
- Inertial element

### 2.4.1 Visual element

```xml
<visual>
  <origin xyz="0 0 0" rpy="0 0 0"/>
  <geometry>
    <mesh filename="package://abb_irb120_description/meshes/visual/link_1.stl"/>
  </geometry>
</visual>
```

Key components:
- `<origin xyz="" rpy="">`
  - Transform from the link frame to the mesh frame
  - `xyz` → translation (meters)
  - `rpy` → roll, pitch, yaw (radians)
- `<geometry>`
  - Can be:
    - `<box>`
    - `<cylinder>`
    - `<sphere>`
    - `<mesh>`
- `<mesh filename="package://...">`
  - `package://` resolves using ROS package indexing
  - Mesh files are usually:
    - `.stl`
    - `.obj`
    - `.dae`
  - Visual meshes are often high-resolution


### 2.4.2 Collision element

```xml
<collision>
  <origin xyz="0 0 0" rpy="0 0 0"/>
  <geometry>
    <mesh filename="package://abb_irb120_description/meshes/collision/link_1.stl"/>
  </geometry>
</collision>
```

Collision geometry is typically:
- Simplified
- Lower-resolution
- Faster for collision checking

**Important**:
- Visual ≠ Collision.
- Collision meshes are optimized for computation.


### 2.4.3 Inertial Element

```xml
<inertial>
  <origin xyz="0 0 0" rpy="0 0 0"/>
  <mass value="3.2"/>
  <inertia ixx="..." ixy="..." ixz="..."
           iyy="..." iyz="..."
           izz="..."/>
</inertial>
```

This defines:
- Mass (kg)
- 3×3 inertia tensor

RViz does not use this, but physics engines (Gazebo, Isaac, etc.) do for simulation.

## 2.5 Joints

A `<joint>` defines how two links connect.

```xml
<joint name="joint_1" type="revolute">
  <parent link="base_link"/>
  <child link="link_1"/>
  <origin xyz="0 0 0.29" rpy="0 0 0"/>
  <axis xyz="0 0 1"/>
  <limit lower="-3.14" upper="3.14"
         effort="100" velocity="1.5"/>
</joint>
```


### 2.5.1 Joint types

Common types:
- `fixed`
- `revolute` (bounded rotation)
- `continuous` (unbounded rotation)
- `prismatic` (linear motion)
- `floating` (6-DOF)
- `planar`

For industrial arms like the ABB IRB 120, all joints are typically **revolute**.


### 2.5.2 Parent / Child
URDF forms a **tree**, not a graph.

Each joint defines:
- A parent link
- A child link

This builds the full kinematic chain:

```txt
base_link → joint_1 → link_1
link_1    → joint_2 → link_2
...
```


### 2.5.3 Origin
```xml
<origin xyz="x y z" rpy="r p y"/>
```

This defines the transform from the parent link frame to the joint frame. This is critical for defining the robot's kinematic geometry and computing forward kinematics (more on this next week).


### 2.5.4 Axis
```xml
<axis xyz="0 0 1"/>
```

Defines the axis of motion in the joint frame.
- For revolute joints, this is the rotation axis.
- For prismatic joints, this is the translation direction.

### 2.5.5 Limits
```xml
<limit lower="..." upper="..."
       effort="..." velocity="..."/>
```

Defines:
- Joint bounds (radians or meters)
- Max effort (Nm or N)
- Max velocity

These values connect directly to:
- Configuration space constraints
- Motion planning constraints

## 2.6 Frames in URDF
In addition to physical links with geometry, URDF often defines pure reference frames. These are implemented as links with no visual or collision geometry, connected by fixed joints.

These frames are critical for:
- Kinematics
- Tool definition
- Calibration
- Motion planning
- Integration with external systems

Example reference frame:

```xml
<link name="base"/>
```

No `<visual>`, `<collision>`, or `<inertial>` elements are required.

It is then attached using a fixed joint:

```xml
<joint name="base_link-base" type="fixed">
  <origin xyz="0 0 0" rpy="0 0 0"/>
  <parent link="base_link"/>
  <child link="base"/>
</joint>
```

This creates a rigid transform between `base_link` and `base`.

---

# 3. RViz2

RViz2 is ROS 2’s primary 3D visualization tool. It allows you to visualize:
- Robot models (URDF)
- Coordinate frames (TF)
- Sensor data (point clouds, laser scans, images)
- Markers and debugging visuals
- Planning results

RViz does **not** compute kinematics or physics. It is purely a visualization tool that renders data being published in the ROS graph.

For robot models specifically:
- URDF defines the structure and geometry.
- `/joint_states` defines the configuration.
- `/tf` defines the transforms between frames.
- RViz renders the robot in 3D using this information.

You can think of RViz as a live window into the current state of the ROS system.

## 3.1 Setup
To visualize a robot, RViz2 typically needs:
- TF transforms (published by `robot_state_publisher`)
- joint states (`/joint_states`) (e.g., from `joint_state_publisher_gui`)
- the URDF string on the `robot_description` parameter

Common minimal setup:
- `robot_state_publisher` (reads URDF, publishes TF)
- `joint_state_publisher_gui` (publishes `/joint_states`)
- `rviz2` (visualizes)


## 3.2 Launch the robot visualization
Run the provided launch file:

```bash
ros2 launch abb_irb120_description display.launch.py
```

To load the robot model:
1. Click **Add** in the bottom left
2. Select **RobotModel** then **OK**
3. Expand the details for the RobotModel in the **Displays** panel, and set **Description Topic** to `/robot_description`
   - Clicking on the empty space next to Description Topic should show an arrow for a drop down list that you can use to select the topic
4. Change the **Fixed Frame** (under Global Options) to a relevant frame, e.g., `base`.

After these steps, you should see that the robot is mostly a jumble of parts centered at the origin. This is because the visualization does not know what configuration to show the robot with. RViz2 subscribes to the robot configuration (state) over the topic `/joint_states`.

We will show two methods of publishing methods to `/joint_states`, first using a GUI tool with manual sliders, and then programmatically using ROS Python.

Important:
- Only run **one** thing publishing `/joint_states` at a time.

## 3.3 Jogging joints with joint_state_publisher_gui
In a separate terminal:

```bash
ros2 run joint_state_publisher_gui joint_state_publisher_gui
```

Move sliders to change joint values and watch the robot in RViz.

# 3.4 Publishing joint states with rclpy

An example script is provided in `workshops/Session08/joint_state_ramp.py`, which moves a joint on the robot back and forth between fixed limits and at a fixed speed.

Run this script directly and you should see the robot's first joint rotating back and forth.

**Quick exercise**: Copy the script into our new `arc380_session08` package and create a new entry point that runs the script using `ros2 run arc380_session08 joint_state_ramp`.

---

# 4. ROS Parameters

A **parameter** is a named value stored on a node  used to configure behavior.

Why parameters are useful:
- Change node behavior without editing code
- Tune values while the node is running (great for demos + debugging)
- Configure nodes from launch files / YAML
- Make scripts reusable across different robots (different joint names, limits, rates, etc.)

**Code along**: Convert `speed` in the `JointStateRamp` node to a parameter.

```python
from rclpy.parameter import Parameter
from rcl_interfaces.msg import SetParametersResult

class JointStateRamp(Node):
    def __init__(self):
        ...

        # Declare a parameter, then get its value
        self.declare_parameter("speed", 0.5)    # speed is in rad/s
        self.speed = float(self.get_parameter("speed").value)

        # Set up a callback so that we can update speed if the parameter value changes
        # Only needs to be defined once per node
        self.add_on_set_parameters_callback(self._on_param_set)

        ...

    def _on_param_set(self, params: list[Parameter]) -> SetParametersResult:
            """Allow updating parameters while the node is running."""
            for p in params:
                if p.name == "speed":
                    new_speed = float(p.value)
                    # Defensive programming to make sure we don't get negative speeds
                    if new_speed <= 0.0:
                        return SetParametersResult(successful=False, reason="speed must be > 0")
                    # Update the internal value of speed
                    # (This is what is used in the control loop)
                    self.speed = new_speed
                    # Optionally log that the speed was updated
                    self.get_logger().info(f"Updated speed = {self.speed:.3f} rad/s")
                # You can add additional parameter updates here, e.g.:
                # if p.name == "joint_name":
                #     ...
            return SetParametersResult(successful=True)
```


There are three primary ways to update parameters:
- Using the command line interface: `ros2 param set ...`
- Using a GUI: `rqt` → Plugins → Configuration → Dynamic Reconfigure
  - On Mac, this plugin may not be directly visible. Instead, you may need to run `ros2 run rqt_reconfigure rqt_reconfigure`
- Using `rclpy`: calling the `/<node_name>/set_parameters` service ([SetParameters](https://docs.ros2.org/foxy/api/rcl_interfaces/srv/SetParameters.html) message type)

`rqt` is the simplest method:
- Run your updated `JointStateRamp` node (either run the script directly or rebuild/source/`ros2 run` if part of a package)
- Run `rqt` and open the Dynamic Reconfigure plugin (or use `ros2 run rqt_reconfigure rqt_reconfigure` on Mac)
- Select our `JointStateRamp` in the node list (refresh the list if you do not see it), then the `speed` parameter should appear
- Try changing the value for `speed`. You should see the speed of the robot change in RViz after changing.

**Exercise**: Convert the other hard-coded values into configurable parameters:
- `joint_name`
- `min_value`
- `max_value`

---

# 5. Troubleshooting checklist

## `JointStateRamp` script does not run while RViz is open
More specifically, the script freezes on running and never logs something like:
```txt
[INFO] [1771709236.675564300] [joint_state_ramp]: Publishing /joint_states: moving 'joint_1' in [-1.00, 1.00] rad at 0.50 rad/s
```
Close out of all of your currently running ROS scripts (including RViz), then try restarting the ROS daemon:

```bash
ros2 daemon stop
ros2 daemon start
```

Then try relaunching the display launcher and running the `JointStateRamp` script.

## Cannot open both the RViz and rqt GUI at the same time
This might occur when trying to run both from an integrated terminal in VS Code.

Try opening separate system terminals (not in VS Code) and running RViz and rqt from those terminals.

## “Package not found” / “executable not found”
- Did you run `colcon build` from the workspace root?
- Did you source the workspace in this terminal?

```bash
call install/setup.bat      # cmd (Windows command prompt)
source install/setup.bash   # bash (MacOS/Linux bash terminal)
source install/setup.zsh    # zsh (MacOS zsh terminal)
```

## Robot doesn’t move in RViz
- Ensure only ONE publisher on `/joint_states`:
  - stop `joint_state_publisher_gui` if your script is running (or vice versa)
- Check that messages exist:

```bash
ros2 topic echo /joint_states --once
```

## RViz shows nothing
- Check RViz **Fixed Frame**
- Ensure `robot_state_publisher` is running and publishing TF:

```bash
ros2 topic echo /tf --once
```


# 6. References + Additional Resources

- ROS 2 command line tools: `ros2 -h`, `ros2 topic -h`, `ros2 pkg -h`
- ROS 2 URDF Documentation + Tutorials: https://docs.ros.org/en/jazzy/Tutorials/Intermediate/URDF/URDF-Main.html
- ROS 2 RViz2 Documentation: https://docs.ros.org/en/jazzy/Tutorials/Intermediate/RViz/RViz-Main.html


# 7. Assignments

- Quiz 5: Robot Terminology + Configuration Space (due Sun, Feb 22, 11:59 pm)
- Assignment 3: Configuration-Space Waypoint Player (deadline to be announced via Canvas/Ed)
- No new quiz for this workshop